<a href="https://colab.research.google.com/github/Geovanecost/TrabalhoN1_Tema02-T-tulo-Gest-o-de-Planejamento-e-dimensionamento-de-esta-es-de-coleta-de-RS/blob/main/Trabalho_N1_Semin%C3%A1rio_de_Algor%C3%ADtmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import math

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO _______________________________________________________________________________

# Solicitação ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação aos nomes dos bairros e inserção na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA E ARMAZENAMENTO

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal = []

for i, bairro in enumerate(nomes_bairros):
    # Total de habitantes no bairro atual
    n_hab = n_hab_bairro[i]

    # Quantidade de resíduos gerados semanalmente por um habitante
    residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Cálculo da quantidade de resíduos para o bairro
    residuos_bairro_semanal = round(n_hab * residuos_por_habitante_semanal)

    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

# Volume necessário por m³ para armazenar os resíduos nos depósitos:
volumes_necess_metrocub = []
peso_por_metro_cubico_necessario = 631  # kg que correspondem a 1 m³

for residuos in residuos_por_bairro_semanal:
    vol_necess_metrocub = residuos / peso_por_metro_cubico_necessario
    volumes_necess_metrocub.append(vol_necess_metrocub)

# PARTE 3 - DIMENSIONAMENTOS DAS ESTAÇÕES DE COLETA E TRIAGEM

# Dimensões da estação de coleta:
larg_estacao = 20  # metros
compri_estacao = 30  # metros
altura_estacao = 7  # metros

# Dimensões de cada depósito:
larg_dep = 10  # metros
compri_dep = 20  # metros
altura_dep = 7  # metros

# Dimensões da área de triagem:
larg_triagem = 20  # metros
compri_triagem = 10  # metros
altura_triagem = 7  # metros

# Capacidade de cada estação em m³
estacao_m = 7000  # m³

# Definição da quantidade de estações por bairro
quantidades_estacoes_por_bairro = []

for i, residuos_bairro_semanal in enumerate(residuos_por_bairro_semanal):
    # Volume total de resíduos a ser armazenado por bairro
    vol_total_bairro = residuos_bairro_semanal / peso_por_metro_cubico_necessario

    # Quantidade de estações necessárias para o bairro
    quantidade_estacao = math.ceil(vol_total_bairro / estacao_m)
    quantidades_estacoes_por_bairro.append(quantidade_estacao)

# Impressão dos resultados por bairro e dimensionamento das estações
print()
print('::::::::::::::::::::::::::::::::::::::: DIMENSIONAMENTOS ::::::::::::::::::::::::::::::::::::::::::')
print()
print('=-=' * 35)
print()

for i, bairro in enumerate(nomes_bairros):
    print(f'Bairro: {bairro}')
    print(f'Residuos semanais por bairro: {residuos_por_bairro_semanal[i]} kg')
    print(f'Número de habitantes: {n_hab_bairro[i]}')
    print()
    print('=-=' * 35)
    print()
    print('Dimensões da Estação de Coleta e Triagem:')
    print(f'Largura: {larg_estacao} m')
    print(f'Comprimento: {compri_estacao} m')
    print(f'Área: {larg_estacao * compri_estacao} m²')
    print(f'Altura: {altura_estacao} m')
    print()
    print('=-=' * 35)
    print()
    print('Dimensões do Depósito:')
    print(f'Largura: {larg_dep} m')
    print(f'Comprimento: {compri_dep} m')
    print(f'Área: {larg_dep * compri_dep} m²')
    print(f'Altura: {altura_dep} m')
    print()
    print('=-=' * 35)
    print()
    print('Dimensões do Setor de Triagem:')
    print()
    print(f'Largura: {larg_triagem} m')
    print(f'Comprimento: {compri_triagem} m')
    print(f'Área: {larg_triagem * compri_triagem} m²')
    print(f'Altura: {altura_triagem} m')
    print()
    print('=-=' * 35)
    print(f'Quantidade de Estações de Coleta e Triagem necessárias para o bairro: {quantidades_estacoes_por_bairro[i]} unidades')
    print()
    print(':' * 105)
    print()


Informe o nome da cidade: Serra da Saudade
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 831
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: Centro
Digite o número de habitantes do bairro Centro: 400
Digite o nome do 2º bairro: São Geraldo
Digite o número de habitantes do bairro São Geraldo: 431

::::::::::::::::::::::::::::::::::::::: DIMENSIONAMENTOS ::::::::::::::::::::::::::::::::::::::::::

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Bairro: Centro
Residuos semanais por bairro: 100 kg
Número de habitantes: 400

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Dimensões da Estação de Coleta e Triagem:
Largura: 20 m
Comprimento: 30 m
Área: 600 m²
Altura: 7 m

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Dimensões do Depósito:
Largura: 10 m
Comprimento

O cálculo da quantidade de estações leva em consideração o volume total necessário para armazenar os resíduos de cada bairro e a capacidade de uma estação (estacao_m). Assim, o código fornece a quantidade e dimensões de estações de coleta e triagem para cada bairro, ajudando no planejamento mais eficiente das infraestruturas de gestão de resíduos. Foi incluso o espaço adicional de 10% da área dos depósitos para o movimento dos trabalhadores, você deve ajustar o cálculo das dimensões dos depósitos

In [ ]:
import math

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO _______________________________________________________________________________

# Solicitação ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação aos nomes dos bairros e inserção na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA E ARMAZENAMENTO

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal = []

for i, bairro in enumerate(nomes_bairros):
    # Total de habitantes no bairro atual
    n_hab = n_hab_bairro[i]

    # Quantidade de resíduos gerados semanalmente por um habitante
    residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Cálculo da quantidade de resíduos para o bairro
    residuos_bairro_semanal = round(n_hab * residuos_por_habitante_semanal)

    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

# Volume necessário por m³ para armazenar os resíduos nos depósitos:
peso_por_metro_cubico_necessario = 631  # kg que correspondem a 1 m³

# PARTE 3 - DIMENSIONAMENTOS DAS ESTAÇÕES DE COLETA E TRIAGEM

# Dimensões da estação de coleta:
larg_estacao = 20  # metros
compri_estacao = 30  # metros
altura_estacao = 7  # metros

# Dimensões de cada depósito:
larg_dep = 10  # metros
compri_dep = 20  # metros
altura_dep = 7  # metros

# Ajuste para o espaço de 10% adicional para o movimento dos trabalhadores
espaco_movimento = 0.10  # 10%
larg_dep_ajustado = larg_dep * (1 + espaco_movimento)
compri_dep_ajustado = compri_dep * (1 + espaco_movimento)
volume_dep_ajustado = larg_dep_ajustado * compri_dep_ajustado * altura_dep

# Dimensões da área de triagem:
larg_triagem = 20  # metros
compri_triagem = 10  # metros
altura_triagem = 7  # metros

# Capacidade de cada estação em m³
estacao_m = 7000  # m³

# Definição da quantidade de estações por bairro
quantidades_estacoes_por_bairro = []

for i, residuos_bairro_semanal in enumerate(residuos_por_bairro_semanal):
    # Volume total de resíduos a ser armazenado por bairro
    vol_total_bairro = residuos_bairro_semanal / peso_por_metro_cubico_necessario

    # Quantidade de depósitos necessários (ajustado para o espaço de movimento)
    quantidade_depositos = math.ceil(vol_total_bairro / volume_dep_ajustado)

    # Quantidade de estações necessárias para o bairro
    quantidade_estacao = math.ceil(vol_total_bairro / estacao_m)
    quantidades_estacoes_por_bairro.append((quantidade_estacao, quantidade_depositos))

# Impressão dos resultados por bairro e dimensionamento das estações
print()
print('::::::::::::::::::::::::::::::::::::::: DIMENSIONAMENTOS ::::::::::::::::::::::::::::::::::::::::::')
print()
print('=-=' * 35)
print()

for i, bairro in enumerate(nomes_bairros):
    quantidade_estacao, quantidade_depositos = quantidades_estacoes_por_bairro[i]
    print(f'Bairro: {bairro}')
    print(f'Resíduos semanais por bairro: {residuos_por_bairro_semanal[i]} kg')
    print(f'Número de habitantes: {n_hab_bairro[i]}')
    print()
    print('=-=' * 35)
    print()
    print('Dimensões da Estação de Coleta e Triagem:')
    print(f'Largura: {larg_estacao} m')
    print(f'Comprimento: {compri_estacao} m')
    print(f'Área: {larg_estacao * compri_estacao} m²')
    print(f'Altura: {altura_estacao} m')
    print()
    print('=-=' * 35)
    print()
    print('Dimensões do Depósito (ajustado para 10% de espaço de movimento):')
    print(f'Largura: {larg_dep_ajustado} m')
    print(f'Comprimento: {compri_dep_ajustado} m')
    print(f'Área: {larg_dep_ajustado * compri_dep_ajustado} m²')
    print(f'Altura: {altura_dep} m')
    print()
    print('=-=' * 35)
    print()
    print('Dimensões do Setor de Triagem:')
    print()
    print(f'Largura: {larg_triagem} m')
    print(f'Comprimento: {compri_triagem} m')
    print(f'Área: {larg_triagem * compri_triagem} m²')
    print(f'Altura: {altura_triagem} m')
    print()
    print('=-=' * 35)
    print(f'Quantidade de Estações de Coleta e Triagem necessárias para o bairro: {quantidade_estacao} unidades')
    print(f'Quantidade de Depósitos necessários para o bairro (ajustado para o espaço de movimento): {quantidade_depositos} unidades')
    print()
    print(':' * 105)
    print()
